In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.font_manager

#loads in all three datasets
fires = pd.read_csv("data/fires.csv")
chip = pd.read_csv("data/CrimeRate.csv")
humps = pd.read_csv("data/speedhumps.csv")



In [3]:
#creates dictionaries for each metric

#fires dictionary
fire_count = {}

for index, row in fires.iterrows():
    neighborhood = row['neighborhood']
    if neighborhood in fire_count:
        fire_count[neighborhood] += 1
    elif neighborhood not in fire_count and neighborhood != None:
        fire_count[neighborhood] = 1

#crime dictionary
dict = {}
dict_count = {}
for index, row in chip.iterrows():
    if not row['INCIDENTNEIGHBORHOOD'] in dict:
        dict[row['INCIDENTNEIGHBORHOOD']] = row['HIERARCHY']
        dict_count[row['INCIDENTNEIGHBORHOOD']] = 1
    else:
        dict[row['INCIDENTNEIGHBORHOOD']] = dict.get(row['INCIDENTNEIGHBORHOOD']) + row['HIERARCHY']
        dict_count[row['INCIDENTNEIGHBORHOOD']] = dict_count.get(row['INCIDENTNEIGHBORHOOD']) + 1
        
mean = {}
for (k,v), (k2,v2) in zip(dict.items(), dict_count.items()):
     mean[k] = v/v2


#humps dictionary
hph = {}
hoods = []
nums = []

for index, row in humps.iterrows():
    hood = row["neighborhood"]
    if hood in hph:
        hph[hood] += 1
    else:
        hph[hood] = 1
        
for i in hph:
    hoods.append(i)
    nums.append(hph[i])
    
hph = {"Neighborhood":hoods, "Humps":nums}


In [4]:
#turns our dictionaries into dataframes for easier manipulation

#fires dataframe
graph = pd.DataFrame.from_dict(fire_count, orient='index', dtype = None, columns=None)
graph = graph.iloc[1: , :]

#crime dataframes
count_df = pd.DataFrame.from_dict(dict_count, orient = 'index')
mean_dataframe = pd.DataFrame.from_dict(mean, orient = 'index')

#humps dataframe
graphHumps = pd.DataFrame.from_dict(hph)

